In [ ]:
!pip install nltk
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Load the data
data = pd.read_csv("final_dataset (1).csv")

# Function to split text into chunks
def split_into_chunks(text, max_words=300, overlap=50):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_words = 0

    for sentence in sentences:
        words = sentence.split()
        if current_words + len(words) > max_words:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-overlap:]  # Retain overlap
            current_words = len(current_chunk)
        current_chunk.extend(words)
        current_words += len(words)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [ ]:
# Apply chunking to the combined text
data['combined_text'] = data['Course_Name'] + " " + data['Course_Link'] + " " + data['Course_Description'] + " " + data['Course_Curriculum']
data['chunks'] = data['combined_text'].apply(lambda x: split_into_chunks(x))

In [ ]:
data['combined_text']

,combined_text
0,"Gemini: Google's AI That Reads , Sees, Hears a..."
1,Reimagining GenAI: Common Mistakes and Best Pr...
2,Coding a ChatGPT-style Language Model from Scr...
3,Mastering Multilingual GenAI Open-Weights for ...
4,Learning Autonomous Driving Behaviors with LLM...
...,...
58,Tableau for Beginners /courses/tableau-for-beg...
59,Getting Started with Neural Networks /courses/...
60,Introduction to AI & ML /courses/introduction-...
61,Winning Data Science Hackathons - Learn from E...


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare data for embedding generation
embeddings = []
metadata = []

for index, row in data.iterrows():
    for chunk in row['chunks']:
        # Generate embedding for each chunk
        embedding = model.encode(chunk)
        embeddings.append(embedding)

        # Store metadata
        metadata.append({
            'Course_Name': row['Course_Name'],
            'Course_Link': row['Course_Link'],
            'Course_Description': row['Course_Description'],
            'Course_Curriculum': chunk
        })





In [ ]:
embeddings

[array([-9.04068071e-03, -1.51565686e-01,  2.79715452e-02, -7.17446655e-02,
         1.27319237e-02, -4.66508418e-02,  3.35397124e-02,  1.45966522e-02,
        -4.66397405e-02, -2.65542977e-02, -8.42879638e-02,  9.23943799e-03,
        -3.39009101e-03, -1.69483442e-02, -1.76627040e-02, -1.04978057e-02,
         2.15994194e-02,  1.04104634e-02, -2.99250297e-02, -8.92992467e-02,
        -2.59059537e-02,  9.60666221e-03,  7.45467171e-02,  1.29116303e-03,
        -3.69806937e-03,  9.52131003e-02, -1.28722498e-02, -7.98829272e-02,
         4.61461432e-02, -6.38107210e-02,  6.25676587e-02,  6.33083358e-02,
         1.86869409e-02,  1.67258605e-02, -9.08118784e-02,  2.21182425e-02,
        -7.16809407e-02, -4.24166583e-02,  1.86360311e-02, -8.17994773e-02,
        -4.41852733e-02, -4.34718281e-02, -1.23435631e-02, -2.79039703e-02,
         1.12234920e-01,  3.39323990e-02, -5.44252805e-02, -2.49006618e-02,
         6.70455396e-02,  2.89818514e-02, -1.22124717e-01, -1.32909998e-01,
        -2.3

In [ ]:
!pip install pinecone-client


In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
import numpy as np

# Initialize Pinecone using the new method
api_key = "pcsk_73PRpV_CgWMY7AHHNkEg9fbvMuLVS2G85A22t8K4QPBw3UBd1ZvmMe472YJzKBB9XQCPX5"  # Your Pinecone API key

# Create an instance of the Pinecone class
pc = Pinecone(api_key=api_key)

# Define the index name
index_name = "drishti"  # Replace with a meaningful name for your index

# Ensure the index does not already exist before creating it
if index_name not in pc.list_indexes().names():
    # Create the index (ServerlessSpec is used for serverless indexes)
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension of your embeddings
        metric='cosine',  # Similarity metric
        spec=ServerlessSpec(cloud='aws', region='us-east-1')  # Serverless index specifications
    )

# Connect to the index
# Use pc.Index instead of pc.index
index = pc.Index(index_name)



In [ ]:
index

In [ ]:
# Upsert the embeddings into the Pinecone index
upsert_data = []

# Prepare the data for upsert (ID, Vector, Metadata)
for i, embedding in enumerate(embeddings):
    vector_id = f"vec{i}"  # Unique ID for each vector (you can customize this based on your use case)
    vector = embedding.tolist()  # Convert the embedding into a list format (Pinecone expects lists of floats)

    # Example metadata associated with each vector (Optional)
    metadata_entry = metadata[i]

    # Add (ID, vector, metadata) to the upsert data
    upsert_data.append((vector_id, vector, metadata_entry))

# Perform the upsert operation (this might take a while depending on the data size)
index.upsert(vectors=upsert_data)



{'upserted_count': 87}

In [ ]:
pip install langchain pinecone-client transformers


In [ ]:
!pip install -U langchain-community

In [ ]:
# Install necessary packages
!pip install langchain pinecone-client transformers

# Install langchain_community
!pip install -U langchain-community

In [ ]:
!pip install langchain_huggingface
import pinecone
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Pinecone
from langchain.embeddings import SentenceTransformerEmbeddings



In [ ]:
!pip install langchain
!pip install transformers
!pip install pinecone-client


In [ ]:
from langchain.llms.base import LLM
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Pinecone
from transformers import pipeline
import pinecone

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
import numpy as np

# Initialize Pinecone using the new method
api_key = "pcsk_73PRpV_CgWMY7AHHNkEg9fbvMuLVS2G85A22t8K4QPBw3UBd1ZvmMe472YJzKBB9XQCPX5"  # Your Pinecone API key

# Create an instance of the Pinecone class
pc = Pinecone(api_key=api_key)

# Define the index name
index_name = "drishti"  # Replace with a meaningful name for your index

# Ensure the index does not already exist before creating it
if index_name not in pc.list_indexes().names():
    # Create the index (ServerlessSpec is used for serverless indexes)
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension of your embeddings
        metric='cosine',  # Similarity metric
        spec=ServerlessSpec(cloud='aws', region='us-east-1')  # Serverless index specifications
    )

In [ ]:
model_name = "google/flan-t5-large"  # Or choose another model such as bart-large
generator = pipeline('text2text-generation', model=model_name)

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install transformers>=4.28.0

In [ ]:
!pip install transformers[torch] sentencepiece

In [ ]:
from typing import Any, Dict, List, Mapping, Optional
from langchain.llms.base import LLM
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM,SequenceFeatureExtractor,TFPreTrainedModel,PreTrainedTokenizerFast,PreTrainedModel # Import necessary modules
from pydantic import Field # Import Field for defining model attributes
import torch

# Define a custom class for LangChain LLM interface using Hugging Face
class HuggingFaceLLM(LLM):
    # Removed default_factory, as it was causing issues
    generator: pipeline = None

    def __init__(self, model_name: str = "google/flan-t5-large"):
        super().__init__()
        # Moved pipeline initialization here to avoid issues with forward references in Pydantic
        self.generator = pipeline('text2text-generation', model=model_name)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.generator(prompt)  # Adding max_length to prevent infinite generation
        return response[0]['generated_text']

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "huggingface" # Add this line to specify the LLM type

# Initialize the model
hf_model = HuggingFaceLLM(model_name="google/flan-t5-large")

In [ ]:
import pandas as pd

# Load the course data
df = pd.read_csv("final_dataset (1).csv")

# Combine relevant columns into a single text field
df['combined_text'] = df['Course_Name'] + "  " + df['Course_Link'] + "  " + df['Course_Description'] + "  " + df['Course_Curriculum']

# Convert the column to a list
texts = df['combined_text'].tolist()

In [ ]:
import os
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer

# Load your SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Assuming you have your texts in a list called 'texts'


texts = [t for t in texts if t is not ...]

# **Set the Pinecone API key as an environment variable**
os.environ["PINECONE_API_KEY"] = "pcsk_73PRpV_CgWMY7AHHNkEg9fbvMuLVS2G85A22t8K4QPBw3UBd1ZvmMe472YJzKBB9XQCPX5"  # Replace with your actual API key
# You may also need to set the environment for your Pinecone environment:
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"  # Replace with your actual environment

# Create and populate the Pinecone vector store
vectorstore = Pinecone.from_texts(
    texts,
    lambda x: model.encode(x).tolist(),
    index_name='drishti',



)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
# Define the prompt template for question-answering
prompt_template = """
You are an assistant that helps users find details of relevant courses.Give answer with proper spacing and next line. Tthere should be no grammatical errors. If user ask about specific details of specific course the
give that. for example- details of course curriculum of specific course. then provide only course curriculum detail. if asks about entire detail then provide course name,link,description and curriculum.
if it asks randomly about Gen AI, data science, machine learning courses,then give top 3 course detail as output.
Context: {context}
Answer the following question: {question}
"""

# Assuming you have PromptTemplate imported
from langchain import PromptTemplate
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

# Assuming you have necessary modules imported
from langchain.chains import RetrievalQA
from langchain.chains.retrieval_qa.base import RetrievalQA

retriever = vectorstore.as_retriever()

# Set up the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=hf_model,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)



# Run the system with a query
query = "list free gen AI courses"

# Provide the query as input
response = qa_chain(query)
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (1161 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'query': 'list free gen AI courses', 'result': 'Generative AI - A Way of Life - Free Course', 'source_documents': [Document(metadata={'Course_Curriculum': "['What is Generative AI', 'How does Gen AI work', 'Quiz', 'Text Generation with Gen AI', 'Image Generation with Gen AI', 'Quiz', 'Meet your Instructors', 'Course Handout', 'Your Feedback Matters!']", 'Course_Description': "This course will provide you with a comprehensive understanding of generative AI, including text and image generation techniques. By the end of the course, you will be have an understanding of using generative AI tools to create diverse content. You'll learn how generative AI works, engage in practical exercises, and gain the skills to implement these techniques in real-world projects.\nWho Should Enroll:\n\n\nProfessionals: Individuals looking to enhance their skills in generative AI and apply advanced techniques to create innovative solutions across various domains.\n\nAspiring Students: Individuals eager to en

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
import streamlit as st
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone
import pandas as pd

# Retrieve API key from environment variables
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("Pinecone API key is not set. Please configure it in the environment variables.")

INDEX_NAME = "drishti"
pc = Pinecone(api_key=PINECONE_API_KEY)

# Ensure the index exists, otherwise create it
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,  # Adjust according to your embeddings' dimension
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-west-2")
    )

# Access the Pinecone index
pc_index = pc.Index(INDEX_NAME)

# Streamlit app
st.title("Smart Search System")
st.markdown("---")

# Default database file (ensure this file exists in the same directory)
DEFAULT_DATABASE = "dataset.csv"

@st.cache_data
def process_file(file):
    """Process the default file and prepare it for vectorstore creation."""
    data = pd.read_csv(file)
    required_columns = {"Course_Name", "Course_Link", "Course_Description", "Course_Curriculum"}
    if not required_columns.issubset(data.columns):
        raise ValueError(f"CSV file must contain these columns: {required_columns}")

    # Fill missing values with placeholders to avoid incomplete metadata
    data.fillna("N/A", inplace=True)

    # Break course curriculum into chunks
    data["chunks"] = data["Course_Curriculum"].apply(
        lambda text: [text[i:i + 200] for i in range(0, len(text), 200)] if isinstance(text, str) else []
    )
    return data

@st.cache_resource
def create_vectorstore(data):
    """Create a Pinecone vectorstore from the course data."""
    model = SentenceTransformer('all-MiniLM-L6-v2')
    metadata = []
    texts = []

    for _, row in data.iterrows():
        for chunk in row['chunks']:
            texts.append(chunk)
            metadata.append({
                'Course_Name': row.get('Course_Name', 'N/A'),
                'Course_Link': row.get('Course_Link', 'N/A'),
                'Course_Description': row.get('Course_Description', 'N/A'),
                'Course_Curriculum': chunk
            })

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    vectorstore = LangchainPinecone.from_texts(
        texts=texts,
        embedding=embeddings.embed_query,
        index_name=INDEX_NAME,
        metadatas=metadata
    )
    return vectorstore

# Load and process the default database
try:
    processed_data = process_file(DEFAULT_DATABASE)

    # Data Preview with Customizable Row Limit
    st.write("**Preview Course Data**")
    num_rows = st.slider("Number of rows to display", min_value=5, max_value=len(processed_data), value=5, step=1)
    st.write(processed_data.head(num_rows))

    vectorstore = create_vectorstore(processed_data)
    st.success("Vectorstore created successfully!")

    query = st.text_input("Search courses")
    if query:
        # Check if user explicitly requests multiple courses
        user_wants_multiple = any(
            keyword in query.lower() for keyword in ["courses", "multiple", "list", "top"]
        )

        # Determine the number of results to fetch
        k = 3 if user_wants_multiple else 1  # Default to 1 for the best course

        # Perform similarity search
        search_results = vectorstore.similarity_search_with_score(query, k=k)

        if search_results:
            seen = set()
            for result, score in search_results:
                metadata = result.metadata
                # Ensure unique results
                if metadata.get('Course_Name') not in seen:
                    seen.add(metadata.get('Course_Name'))
                    st.write(f"**Course Name:** {metadata.get('Course_Name', 'N/A')}")
                    st.write(f"**Course Link:** {metadata.get('Course_Link', 'N/A')}")
                    st.write(f"**Course Description:** {metadata.get('Course_Description', 'N/A')}")
                    st.write(f"**Course Curriculum:** {metadata.get('Course_Curriculum', 'N/A')}")
                    st.markdown("---")
        else:
            st.warning("No relevant results found.")
except Exception as e:
    st.error(f"An error occurred: {e}")